# Zillow Prediction

## A) Get the data

### 1) List data needed & how much space to use

We are given five files:
1. properties_2016.csv - all the properties with their home features for 2016. Note: Some 2017 new properties don't have any data yet except for their parcelid's. Those data points should be populated when properties_2017.csv is available.
2. properties_2017.csv - all the properties with their home features for 2017 (will be available on 10/2/2017)
3. train_2016.csv - the training set with transactions from 1/1/2016 to 12/31/2016
4. train_2017.csv - the training set with transactions from 1/1/2017 to 9/15/2017 (will be available on 10/2/2017)
5. sample_submission.csv - a sample submission file in the correct format

For now, the data is contained in:
1. properties_2016.csv 
2. train_2016.csv

Thus, we will work only with those two guys.

### 2) Get the data

In [1]:
import pandas as pd

In [2]:
#raw_train = pd.read_csv("Data/train_2016_v2.csv")

In [3]:
#raw_train.info()

In [4]:
#raw_properties = pd.read_csv("Data/properties_2016.csv",error_bad_lines=False, index_col=False, dtype='unicode')

In [5]:
#raw_properties.info()

Here, I want to do a inner join, the properties file contains a lot of data and a lot of observations may be useless.

Because my computer is so bad, I had to do an inner join on train.csv at the linux command line

In [32]:
join_raw = pd.read_csv("Data/joined_data.csv",
                       error_bad_lines=False, 
                       index_col=False, 
                       dtype='unicode', 
                       keep_default_na=False)

In [36]:
join_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90275 entries, 0 to 90274
Data columns (total 60 columns):
parcelid                        90275 non-null int64
logerror                        90275 non-null float64
transactiondate                 90275 non-null datetime64[ns]
airconditioningtypeid           28781 non-null float64
architecturalstyletypeid        261 non-null float64
basementsqft                    43 non-null float64
bathroomcnt                     90275 non-null float64
bedroomcnt                      90275 non-null float64
buildingclasstypeid             16 non-null float64
buildingqualitytypeid           57364 non-null float64
calculatedbathnbr               89093 non-null float64
decktypeid                      658 non-null float64
finishedfloor1squarefeet        6856 non-null float64
calculatedfinishedsquarefeet    89614 non-null float64
finishedsquarefeet12            85596 non-null float64
finishedsquarefeet13            33 non-null float64
finishedsquarefeet15

### 3) Convert the data into good formats

In [ ]:
def createDummiesDateVar(data_Date, _dateVar):
    from datetime import datetime
    data_Date[_dateVar] = data_Date[_dateVar].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))
    data_Date[_dateVar+"_date"] = [d.date() for d in data_Date[_dateVar]]
    data_Date[_dateVar+'_weekdays'] = [d.strftime('%a') for d in data_Date[_dateVar]]
    data_Date = pd.concat([data_Date,pd.get_dummies(data_Date[_dateVar+'_weekdays'])], axis=1)
    data_Date.drop(_dateVar+'_weekdays', axis=1,inplace=True)
    return data_Date

In [34]:
def convertorZillow(data_df):
    # Set everything possible as integer/float
    data_df=data_df.apply(pd.to_numeric, errors='ignore')
    # Set te date into date format
    data_df['transactiondate'] = pd.to_datetime(data_df['transactiondate'])
    data_df=createDummiesDateVar(data_df,'transactiondate')
    # Three boolean variables
    data_df["hashottuborspa"]=data_df["hashottuborspa"].astype('bool')
    data_df["fireplaceflag"]=data_df["fireplaceflag"].astype('bool')
    data_df["taxdelinquencyflag"]=data_df["taxdelinquencyflag"].astype('bool')
    # Two string variables 
    data_df["propertycountylandusecode"]=data_df["propertycountylandusecode"].astype(str)
    data_df["propertyzoningdesc"]=data_df["propertyzoningdesc"].astype(str)
    return data_df

In [35]:
join_raw=convertorZillow(join_raw)

### 4) Sample a test set, put it aside and never look at it again

In [10]:
from sklearn.model_selection import train_test_split

In [37]:
train_set, test_set = train_test_split(join_raw, 
                                       test_size=0.2, 
                                       random_state=42)

In [12]:
del join_raw # RAM optimizer

In [13]:
import gc
gc.collect()

214

## B) Explore data to gain insight

### 1) Study each attribute

#### a) Check names, types and origins of variables

In [14]:
train_set.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
89031,17262590,-0.0101,2016-02-24,NaN,NaN,NaN,3.0,4.0,NaN,NaN,...,2.0,False,140000.0,398000.0,2015,258000.0,4484.82,False,NaN,6.111008e+13
37767,12206224,0.2639,2016-01-22,1.0,NaN,NaN,3.0,3.0,NaN,7.0,...,NaN,False,99129.0,130426.0,2015,31297.0,1626.66,False,NaN,6.037238e+13
446,10728095,-0.0131,2016-05-04,1.0,NaN,NaN,2.0,3.0,NaN,7.0,...,NaN,False,77390.0,100003.0,2015,22613.0,6332.96,False,NaN,6.037135e+13
5088,10877921,-0.0060,2016-07-15,NaN,NaN,NaN,4.0,8.0,NaN,7.0,...,NaN,False,114942.0,171461.0,2015,56519.0,2532.87,False,NaN,6.037123e+13
52321,12849787,-0.0080,2016-04-14,NaN,NaN,NaN,1.0,3.0,NaN,7.0,...,NaN,False,52254.0,261274.0,2015,209020.0,3410.51,False,NaN,6.037408e+13


Ok, seems there is a lot of "NaN" values, we have to handle this

#### b)  Missings values

In [15]:
#train_set.info()

The train_set.info() command shows that some variables only have 27 or 29 non-null observations. 

That's a lot of missing values

Let's drop for now variables with too much missing values, filling them will makes no sense

In [16]:
# Drop column with more than (72220-70000)=2220 NaN values 
train_set_nona=train_set.dropna(axis=1, 
                                how='any', 
                                thresh=70000, 
                                subset=None, 
                                inplace=False)

In [17]:
#train_set_nona.info()

From 60 variables, we have now 29, fair enough. 

Now let's fill missing values with the median because for now, no outliers study have been made

(Todo: do a better filling with clustering)

In [18]:
def medianFiller(data_df):
    # Create a median inputer
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN',strategy='median', axis=1)
    return pd.DataFrame(imp.fit_transform(data_df),columns=data_df.columns.values)

In [19]:
# Get non-numerical variables
non_num_var=["transactiondate",
             "hashottuborspa",
             "propertycountylandusecode",
            "propertyzoningdesc",
            "fireplaceflag",
            "taxdelinquencyflag"]

In [20]:
# Drop non-numerical variables
train_set_nona_filled=train_set_nona.drop(non_num_var, axis=1, inplace=False)

In [21]:
# Add non numerical values
train_clean=pd.concat([train_set_nona_filled,
                       train_set_nona[non_num_var]])
# train_clean is the new training set

In [22]:
# Clean my RAM
del train_set_nona_filled
del train_set_nona
gc.collect()

NameError: name 'imp' is not defined

#### c) Noisiness and type of noise (stochastic, outliers, rounding error, etc..)

Let's delete rows with the same value in each row

In [23]:
cols = list(train_clean)
nunique = train_clean.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
train_clean.drop(cols_to_drop, axis=1,inplace=True)
train_clean.drop("yearbuilt", axis=1,inplace=True) # Yearbuilt contains only 2015, but there is a little bug

27 variables, now we need to know our outliers

In [24]:
train_clean.head()

,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,censustractandblock,fips,fireplaceflag,fullbathcnt,hashottuborspa,landtaxvaluedollarcnt,...,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,structuretaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxvaluedollarcnt,transactiondate
89031,3.0,4.0,3.0,1769.0,6.111008e+13,6111.0,NaN,3.0,NaN,258000.0,...,6.111008e+07,27110.0,2061.0,97116.0,6.0,140000.0,4484.82,NaN,398000.0,NaT
37767,3.0,3.0,3.0,1304.0,6.037238e+13,6037.0,NaN,3.0,NaN,31297.0,...,6.037238e+07,12447.0,3101.0,96025.0,0.0,99129.0,1626.66,NaN,130426.0,NaT
446,2.0,3.0,2.0,1738.0,6.037135e+13,6037.0,NaN,2.0,NaN,22613.0,...,6.037135e+07,12447.0,3101.0,96342.0,0.0,77390.0,6332.96,NaN,100003.0,NaT
5088,4.0,8.0,4.0,3300.0,6.037123e+13,6037.0,NaN,4.0,NaN,56519.0,...,6.037123e+07,12447.0,3101.0,96375.0,0.0,114942.0,2532.87,NaN,171461.0,NaT
52321,1.0,3.0,1.0,984.0,6.037408e+13,6037.0,NaN,1.0,NaN,209020.0,...,6.037408e+07,22827.0,3101.0,96492.0,0.0,52254.0,3410.51,NaN,261274.0,NaT


#### d) Type of distributions

#### e) Is variables useful?

### 2) Identify target attribute

### 3) Data viz

### 4) Study correlations

In [ ]:
#corr_matrix=train_clean.corr()
#corr_matrix["logerror"].sort_values(ascending=False)

### 5) Identify tranformations

### 6) Identify extra data useful

### 7) Documentation

## C) Prepare the data to better expose the underlying data patters to ML algorithms

## D) Explore many different models and short-list the best ones

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf)],
    voting='hard')

In [26]:
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144440 entries, 89031 to 15795
Data columns (total 27 columns):
bathroomcnt                     72220 non-null float64
bedroomcnt                      72220 non-null float64
calculatedbathnbr               71251 non-null float64
calculatedfinishedsquarefeet    71666 non-null float64
censustractandblock             71724 non-null float64
fips                            72220 non-null float64
fireplaceflag                   72220 non-null object
fullbathcnt                     71251 non-null float64
hashottuborspa                  72220 non-null object
landtaxvaluedollarcnt           72219 non-null float64
latitude                        72220 non-null float64
logerror                        72220 non-null float64
longitude                       72220 non-null float64
parcelid                        72220 non-null float64
propertycountylandusecode       72220 non-null object
propertylandusetypeid           72220 non-null float64
propertyz

In [27]:
Y=train_clean["logerror"]

In [28]:
X=train_clean.drop("logerror",axis=1,inplace=False)

In [30]:
voting_clf.fit(X, Y)

ValueError: y contains new labels: [ nan  nan  nan ...,  nan  nan  nan]

In [31]:
Y

89031   -0.0101
37767    0.2639
446     -0.0131
5088    -0.0060
52321   -0.0080
61171   -0.0030
77401    0.0040
86657   -0.0243
87327   -0.1098
10608   -0.0111
20665    0.0218
90177   -0.0141
69275   -0.0141
54924    0.0440
29490   -0.0222
58556    0.0050
24947    0.1142
76381    0.0178
1392     0.0030
21111    0.0266
2679     0.0040
70187    0.1249
71758    0.1638
13902    0.0020
44286   -0.0111
39985    0.0208
18886   -0.2549
16174    0.0770
84171    0.0149
31290    0.0139
          ...  
59150       NaN
2747        NaN
18431       NaN
84478       NaN
25658       NaN
71932       NaN
28693       NaN
85305       NaN
53707       NaN
83104       NaN
5311        NaN
67969       NaN
64925       NaN
62955       NaN
59735       NaN
769         NaN
64820       NaN
67221       NaN
41090       NaN
16023       NaN
60263       NaN
44131       NaN
87498       NaN
37194       NaN
82386       NaN
6265        NaN
54886       NaN
76820       NaN
860         NaN
15795       NaN
Name: logerror, dtype: f

## E) Fine-tune best models & combine them into a better solution

## F) Present results